# PIVOTEO Y DESPIVOTEO DE LA BASE DE DATOS

In [1]:
import pandas as pd
import numpy as np

Copia los datos mensuales y luego crea una nueva columna llamada 'lat_long', que combina las coordenadas de latitud y longitud con un formato específico para cumplir con las reglas de MATLAB. Se reemplazan los puntos por 'p' y los signos negativos por 'n'. Después, pivota el DataFrame para organizar los datos de manera que cada combinación única de mes y coordenadas se convierta en una columna, con los valores de la columna 'CANTIDAD' sumados cuando es necesario y los valores faltantes llenados con ceros.

In [7]:
geodf = pd.read_json("Delitos Colombia/DANE/GeoColombia.json") \
    .rename(columns={"Departamento": "DEPARTAMENTO", "Localidad": "MUNICIPIO"}) \
    .drop(columns=["Tipo"])

clean_data = pd.read_csv('Delitos por Departamento/Colombia/Narcotrafico_Colombia.csv')

monthly_data = pd.merge(clean_data, geodf, on=["DEPARTAMENTO", "MUNICIPIO"], how="inner") #\
    #.query("DESCRIPCION_CONDUCTA_CAPTURA == 'ARTÍCULO 376. TRÁFICO. FABRICACIÓN O PORTE DE ESTUPEFACIENTES'")
    
monthly_data['FECHA_HECHO'] = pd.to_datetime(monthly_data['FECHA_HECHO'])

monthly_data = monthly_data \
    .groupby([pd.Grouper(key='FECHA_HECHO', freq='M'), 'DEPARTAMENTO', 'MUNICIPIO', 'LATITUD', 'LONGITUD']) \
    .sum() \
    .reset_index()

monthly_data['MES'] = monthly_data['FECHA_HECHO'].dt.to_period('M')
monthly_data = monthly_data[['MES', 'LATITUD', 'LONGITUD', 'CANTIDAD']]
monthly_data.to_csv('JULIAN.csv', index=False)


C:\Users\monte\AppData\Local\Temp\ipykernel_27192\3615834374.py:13: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .groupby([pd.Grouper(key='FECHA_HECHO', freq='M'), 'DEPARTAMENTO', 'MUNICIPIO', 'LATITUD', 'LONGITUD']) \


In [8]:
print(monthly_data)

           MES   LATITUD   LONGITUD  CANTIDAD
0      2010-01 -3.530059 -70.045137        11
1      2010-01 -3.631323 -70.471990         4
2      2010-01  6.365534 -75.090597         1
3      2010-01  6.032922 -75.708003         4
4      2010-01  6.977789 -74.981239         4
...        ...       ...        ...       ...
30809  2015-12  4.028408 -76.032504        23
30810  2015-12  3.904130 -76.390804         3
30811  2015-12  3.595914 -76.510988         1
30812  2015-12  4.353599 -76.050653         3
30813  2015-12  5.836530 -68.141222         2

[30814 rows x 4 columns]


In [ ]:
# Copiar los datos mensuales
df = monthly_data.copy()

# Crear la columna 'lat_long' con un prefijo y reemplazar el separador para cumplir con las reglas de MATLAB
df['lat_long'] = (
    'loc_' +
    df['LATITUD'].astype(str).str.replace('.', 'p').str.replace('-', 'n') + # Reemplaza '.' con 'p' y '-' con 'n'
    'x' +
    df['LONGITUD'].astype(str).str.replace('.', 'p').str.replace('-', 'n')
)

# Pivotar el DataFrame
df_pivot = df.pivot_table(index='MES', columns='lat_long', values='CANTIDAD ', aggfunc='sum', fill_value=0)
df_pivot.to_csv("resultado_pivot.csv")

Carga un archivo CSV con los datos previamente pivotados y luego deshace el pivot (convierte las columnas 'lat_long' de nuevo en filas). Después, separa la columna 'lat_long' en dos nuevas columnas: 'LATITUD' y 'LONGITUD'. Los valores de estas columnas se ajustan reemplazando caracteres que fueron usados para formatear las coordenadas (como 'n' por '-' y 'p' por '.') para restaurar los valores numéricos originales. También elimina el prefijo 'loc_' y convierte las columnas 'LATITUD' y 'LONGITUD' a tipo numérico. Finalmente, elimina la columna 'lat_long' que ya no es necesaria.

In [ ]:
# Despivotar el DataFrame (convertir las columnas 'lat_long' en filas)
df_pivot = pd.read_csv('resultado_pivot.csv')
# Despivotar el DataFrame
df_unpivoted = pd.melt(df_pivot, id_vars=['MES'], var_name='lat_long', value_name='CANTIDAD')

# Separar la columna 'lat_long' en 'LATITUD' y 'LONGITUD'
df_unpivoted[['LATITUD', 'LONGITUD']] = df_unpivoted['lat_long'].str.split('x', expand=True)

# Reemplazar 'n' con '-' y 'p' con '.' para recuperar los valores numéricos originales
df_unpivoted['LATITUD'] = df_unpivoted['LATITUD'].str.replace('n', '-', regex=False).str.replace('p', '.', regex=False)
df_unpivoted['LONGITUD'] = df_unpivoted['LONGITUD'].str.replace('n', '-', regex=False).str.replace('p', '.', regex=False)

# Eliminar el prefijo 'loc_' de las coordenadas
df_unpivoted['LATITUD'] = df_unpivoted['LATITUD'].str.replace('loc_', '', regex=False)
df_unpivoted['LONGITUD'] = df_unpivoted['LONGITUD'].str.replace('loc_', '', regex=False)

# Convertir las columnas LATITUD y LONGITUD a tipo numérico
df_unpivoted['LATITUD'] = pd.to_numeric(df_unpivoted['LATITUD'])
df_unpivoted['LONGITUD'] = pd.to_numeric(df_unpivoted['LONGITUD'])

# Eliminar la columna 'lat_long' ya que ahora tenemos LATITUD y LONGITUD
df_unpivoted.drop(columns=['lat_long'], inplace=True)
df_unpivoted['CANTIDAD'] = df_unpivoted['CANTIDAD'].round(0).astype(int)
df_unpivoted[df_unpivoted["CANTIDAD"] > 0].to_csv('resultado_unpivot_rounded.csv', index=False)